In [1]:
# TODO
#!{__import__('sys').executable} -m pip install --quiet openstudio

In [2]:
import contextlib
import builtins

@contextlib.contextmanager
def temporary_attr(o, name: str):
    a = builtins.getattr(o, name)
    try: yield
    finally: builtins.setattr(o, name, a)

import os
import sys

@contextlib.contextmanager
def temporary_search_path(*paths):
    with temporary_attr(sys, 'path'):
        setattr(sys, 'path', [str(p) for p in paths])
        try: yield
        finally: pass

In [3]:
import os
import shutil

def find_energyplus():
    return os.path.dirname(
        os.path.realpath(shutil.which('energyplus'))
    )

In [4]:
# TODO
energyplus_path = os.path.expanduser('~/.local/EnergyPlus-23-1-0')
with temporary_search_path(energyplus_path):
    import pyenergyplus as eplus
    import pyenergyplus.api

In [5]:
import packaging
import io
import csv
import collections


class EnergyPlusEMS:
    _ep_api = eplus.api.EnergyPlusAPI()
    assert (
        packaging.version.Version(_ep_api.api_version()) 
            >= packaging.version.Version('0.2')
    )

    def __init__(self):
        pass
    
    def __enter__(self):
        if getattr(self, '_ep_state', None) is not None:
            raise Exception()
        self._ep_state = self._ep_api.state_manager.new_state()
        return self
        
    def __exit__(self, *_exc_args):
        if getattr(self, '_ep_state', None) is None:
            raise Exception()
        self._ep_api.state_manager.delete_state(self._ep_state)

    def _exec(self, *args):
        self._ep_api.state_manager.reset_state(self._ep_state)
        return self._ep_api.runtime.run_energyplus(
            self._ep_state,
            command_line_args=args
        )

    def _available_data(self):
        with io.StringIO(
            self._ep_api.exchange
                .list_available_api_data_csv(self._ep_state)
                .decode()
        ) as f:
            def _ep_csv_reader(f, default_title=None):
                title = default_title
                for row in csv.reader(f):
                    if len(row) == 1:
                        title = row.pop()
                    yield title, row
        
            d = collections.defaultdict(lambda: [])
            for title, row in _ep_csv_reader(f):
                if not row:
                    continue
                d[title].append(row)
        
            colnames = {
                '**ACTUATORS**': ['Type', 'ComponentTypeName', 'ControlTypeName', 'UniqueIDName'],
                '**INTERNAL_VARIABLES**': ['Type', 'DataTypeName', 'UniqueIDName'],
                '**PLUGIN_GLOBAL_VARIABLES**': ['Type', 'Name'],
                '**TRENDS**': ['Type', 'Name'],
                '**METERS**': ['Type', 'Name'],
                '**VARIABLES**': ['Type', 'VarNameOnly', 'KeyNameOnlyUC']
            }
 
            return {title: pd.DataFrame(d[title], columns=colnames[title]) for title in d}
        

In [12]:
# NOTE example
_ = '''
with EnergyPlusEMS() as ep_ems:
    #ep_ems._exec('--help')
    ep_ems._exec(
        '--output-directory', 'build/demo-eplus',
        '--weather', f'{energyplus_path}/WeatherData/USA_FL_Tampa.Intl.AP.722110_TMY3.epw',
        f'{energyplus_path}/ExampleFiles/CoolingTower_VariableSpeed_MultiCell.idf'
    )
'''

In [6]:
import io
import pandas as pd

ep_ems = EnergyPlusEMS().__enter__()

ep_ems._exec(
    # TODO
    '--design-day',
    '--output-directory', 'build/demo-eplus',
    '--weather', f'{energyplus_path}/WeatherData/USA_FL_Tampa.Intl.AP.722110_TMY3.epw',
    f'{energyplus_path}/ExampleFiles/ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf'
)

EnergyPlus Starting
EnergyPlus, Version 23.1.0-87ed9199d4, YMD=2023.09.12 02:28
Initializing Response Factors
Calculating CTFs for "INTERIORFURNISHINGS"
Calculating CTFs for "DROPCEILING"
Calculating CTFs for "INT_WALL"
Calculating CTFs for "EXT_SLAB_8IN_WITH_CARPET"
Calculating CTFs for "INT_SLAB_FLOOR"
Calculating CTFs for "NONRES_ROOF"
Calculating CTFs for "NONRES_EXT_WALL"
Calculating CTFs for "BASEMENT_WALL_EAST_CFACTOR"
Calculating CTFs for "BASEMENT_WALL_SOUTH_CFACTOR"
Calculating CTFs for "BASEMENT_WALL_NORTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_NORTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_SOUTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_WEST_CFACTOR"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combi

EnergyPlus Completed Successfully.


0

In [15]:
df_avail_data = ep_ems._available_data()

In [17]:
df_avail_data_act = df_avail_data.get('**ACTUATORS**')

In [19]:
df_avail_data_act['ComponentTypeName'].unique()

array(['Weather Data', 'Schedule:Compact', 'Schedule:Constant',
       'Material', 'People', 'Lights', 'ElectricEquipment', 'Surface',
       'Zone', 'Zone Infiltration', 'Plant Loop Overall',
       'Supply Side Half Loop', 'Demand Side Half Loop',
       'Demand Side Branch', 'Plant Component Pipe:Adiabatic',
       'Plant Component WaterUse:Connections', 'Supply Side Branch',
       'Plant Component Pump:ConstantSpeed',
       'Plant Component WaterHeater:Mixed',
       'Plant Component Coil:Heating:Water',
       'Plant Component Pump:VariableSpeed',
       'Plant Component Boiler:HotWater',
       'Plant Component HeatExchanger:FluidToFluid',
       'Plant Component Chiller:Electric:ASHRAE205',
       'Plant Component Coil:Cooling:Water',
       'Plant Component Coil:Cooling:WaterToAirHeatPump:EquationFit',
       'Plant Component Coil:Heating:WaterToAirHeatPump:EquationFit',
       'Plant Component FluidCooler:TwoSpeed',
       'Plant Component HeaderedPumps:VariableSpeed',
     

In [20]:
df_avail_data_act[df_avail_data_act['ComponentTypeName'] == 'Plant Component Chiller:Electric:ASHRAE205']

,Type,ComponentTypeName,ControlTypeName,UniqueIDName
1424,Actuator,Plant Component Chiller:Electric:ASHRAE205,On/Off Supervisory,COOLSYS1 CHILLER1;
1427,Actuator,Plant Component Chiller:Electric:ASHRAE205,On/Off Supervisory,COOLSYS1 CHILLER2;


In [ ]:
df_avail_data_act[df_avail_data_act['ComponentTypeName'] == 'Plant Component CoolingTower:VariableSpeed']

In [22]:
df_avail_data_act[
    df_avail_data_act['ComponentTypeName'].isin(
        ('Zone Temperature Control', 'Zone Temperature Control')
    )
]

,Type,ComponentTypeName,ControlTypeName,UniqueIDName
1631,Actuator,Zone Temperature Control,Heating Setpoint,BASEMENT;
1632,Actuator,Zone Temperature Control,Cooling Setpoint,BASEMENT;
1633,Actuator,Zone Temperature Control,Heating Setpoint,CORE_BOTTOM;
1634,Actuator,Zone Temperature Control,Cooling Setpoint,CORE_BOTTOM;
1635,Actuator,Zone Temperature Control,Heating Setpoint,CORE_MID;
1636,Actuator,Zone Temperature Control,Cooling Setpoint,CORE_MID;
1637,Actuator,Zone Temperature Control,Heating Setpoint,CORE_TOP;
1638,Actuator,Zone Temperature Control,Cooling Setpoint,CORE_TOP;
1639,Actuator,Zone Temperature Control,Heating Setpoint,PERIMETER_BOT_ZN_3;
1640,Actuator,Zone Temperature Control,Cooling Setpoint,PERIMETER_BOT_ZN_3;


In [ ]:
# TODO ep_ems._ep_api.runtime.callback

In [ ]:
ep_ems.__close__()